### 2단계: Azure 서비스 구성

#### 2.1 Azure OpenAI Service (Private Endpoint)

**Azure Portal:**

1.  "Create a resource"에서 "Azure OpenAI"를 검색하여 선택하고 "Create"를 클릭합니다.
2.  다음 정보를 입력합니다.
    -   Subscription: 사용 중인 Azure 구독
    -   Resource group: "secu-ai-workshop"
    -   Region: "Korea Central"
    -   Name: "secu-ai-account"
    -   Pricing tier: Standard S0
3.  "Next: Network"를 클릭하고, "Connectivity type"을 "Private endpoint"로 선택합니다.
4.  "Add private endpoint"를 클릭하고 다음을 수행합니다.
    -   Name: "secu-ai-pe"
    -   Location: "Korea Central"
    -   Virtual Network: spoke1vnet
    -   Subnet: privateendpoint
    -   OK
5.  "Review + create"를 클릭하고 "Create"를 클릭합니다.
6.  Azure OpenAI Service가 배포된 후 gpt-4o 모델 배포
    -   Azure OpenAI Studio로 이동
    -   Deployments에서 Create new deployment 선택
    -   Model: gpt-4o 선택 후 버전 선택,
    -   Deployment name을 입력하고 Create.

In [1]:
# 기존 변수 설정
RESOURCE_GROUP="secu-ai-workshop"
LOCATION="koreacentral"

ONPREM_VNET_NAME="onprem-vnet"
ONPREM_ADDRESS_PREFIX="192.168.0.0/16"
ONPREM_SUBNET_NAME="onprem-subnet"
ONPREM_SUBNET_PREFIX="192.168.1.0/24"
GATEWAY_SUBNET_NAME="GatewaySubnet" # GatewaySubnet 이름은 고정
GATEWAY_SUBNET_PREFIX="192.168.0.0/24"

HUB_VNET_NAME="hubvnet"
HUB_ADDRESS_PREFIX="10.0.0.0/16"
HUB_GATEWAY_SUBNET_PREFIX="10.0.0.0/24"
FIREWALL_SUBNET_NAME="AzureFirewallSubnet" # AzureFirewallSubnet 이름은 고정
FIREWALL_SUBNET_PREFIX="10.0.1.0/26"
DMZ_SUBNET_NAME="dmz"
DMZ_SUBNET_PREFIX="10.0.2.0/24"

SPOKE1_VNET_NAME="spoke1vnet"
SPOKE1_ADDRESS_PREFIX="10.1.0.0/16"
SPOKE1_SUBNET_NAME="spoke1subnet"
SPOKE1_SUBNET_PREFIX="10.1.0.0/24"
SPOKE1_PE_SUBNET_NAME="privateendpoint"
SPOKE1_PE_SUBNET_PREFIX="10.1.1.0/24"

SPOKE2_VNET_NAME="spoke2vnet"
SPOKE2_ADDRESS_PREFIX="10.2.0.0/16"
SPOKE2_SUBNET_NAME="spoke2subnet"
SPOKE2_SUBNET_PREFIX="10.2.0.0/24"

In [4]:
# Azure OpenAI 관련 변수 설정
OPENAI_NAME="secu-ai-account1"
OPENAI_PE_NAME="secu-ai-pe"
OPENAI_PRIVATE_DNS_ZONE="privatelink.openai.azure.com"

In [ ]:
# Azure OpenAI account 생성
!az cognitiveservices account create \
    --name $OPENAI_NAME \
    --custom-domain $OPENAI_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --kind OpenAI \
    --sku s0

{
  "etag": "\"4600433d-0000-2500-0000-67ef4f960000\"",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.CognitiveServices/accounts/secu-ai-account1",
  "identity": null,
  "kind": "OpenAI",
  "location": "koreacentral",
  "name": "secu-ai-account1",
  "properties": {
    "abusePenalty": null,
    "allowedFqdnList": null,
    "apiProperties": {
      "aadClientId": null,
      "aadTenantId": null,
      "additionalProperties": null,
      "eventHubConnectionString": null,
      "qnaAzureSearchEndpointId": null,
      "qnaAzureSearchEndpointKey": null,
      "qnaRuntimeEndpoint": null,
      "statisticsEnabled": null,
      "storageAccountConnectionString": null,
      "superUser": null,
      "websiteName": null
    },
    "callRateLimit": {
      "count": null,
      "renewalPeriod": null,
      "rules": [
        {
          "count": 30.0,
          "dynamicThrottlingEnabled": null,
          "key": "openai.dalle.post",


In [9]:
# Azure OpenAI account의 Private Endpoint 생성
OPENAI_RESOURCE_ID=!az cognitiveservices account show --name $OPENAI_NAME --resource-group $RESOURCE_GROUP --query id -o tsv
OPENAI_RESOURCE_ID=OPENAI_RESOURCE_ID[0].strip()
!echo $OPENAI_RESOURCE_ID

/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.CognitiveServices/accounts/secu-ai-account1


In [10]:

# Private Endpoint 생성
!az network private-endpoint create \
    --name $OPENAI_PE_NAME \
    --resource-group $RESOURCE_GROUP \
    --vnet-name $SPOKE1_VNET_NAME \
    --subnet $SPOKE1_PE_SUBNET_NAME \
    --private-connection-resource-id $OPENAI_RESOURCE_ID \
    --group-ids account \
    --connection-name openai-connection \
    --location $LOCATION


{
  "customDnsConfigs": [
    {
      "fqdn": "secu-ai-account1.openai.azure.com",
      "ipAddresses": [
        "10.1.1.4"
      ]
    }
  ],
  "customNetworkInterfaceName": "",
  "etag": "W/\"c80aee55-e08e-4392-92bd-bab9b0523c8e\"",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/privateEndpoints/secu-ai-pe",
  "ipConfigurations": [],
  "location": "koreacentral",
  "manualPrivateLinkServiceConnections": [],
  "name": "secu-ai-pe",
  "networkInterfaces": [
    {
      "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/networkInterfaces/secu-ai-pe.nic.eb66636c-3236-4fa6-97e4-2b929eecfe0c",
      "resourceGroup": "secu-ai-workshop"
    }
  ],
  "privateLinkServiceConnections": [
    {
      "etag": "W/\"c80aee55-e08e-4392-92bd-bab9b0523c8e\"",
      "groupIds": [
        "account"
      ],
      "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab2

In [11]:
# Private DNS Zone 생성 (privatelink.openai.azure.com)
!az network private-dns zone create \
    --name $OPENAI_PRIVATE_DNS_ZONE \
    --resource-group $RESOURCE_GROUP

{
  "etag": "80aa7ecf-ce0d-49d4-a877-b8fa2d4811ea",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/privateDnsZones/privatelink.openai.azure.com",
  "location": "global",
  "maxNumberOfRecordSets": 25000,
  "maxNumberOfVirtualNetworkLinks": 1000,
  "maxNumberOfVirtualNetworkLinksWithRegistration": 100,
  "name": "privatelink.openai.azure.com",
  "numberOfRecordSets": 1,
  "numberOfVirtualNetworkLinks": 0,
  "numberOfVirtualNetworkLinksWithRegistration": 0,
  "provisioningState": "Succeeded",
  "resourceGroup": "secu-ai-workshop",
  "type": "Microsoft.Network/privateDnsZones"
}


In [12]:
# Private DNS Zone Link 생성 (Spoke1 VNET)
!az network private-dns link vnet create \
    --name spoke1vnetDnsLink \
    --resource-group $RESOURCE_GROUP \
    --zone-name $OPENAI_PRIVATE_DNS_ZONE \
    --virtual-network $SPOKE1_VNET_NAME \
    --registration-enabled false

{
  "etag": "\"9c00d88f-0000-0100-0000-67ef5a7f0000\"",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/privateDnsZones/privatelink.openai.azure.com/virtualNetworkLinks/spoke1vnetdnslink",
  "location": "global",
  "name": "spoke1vnetdnslink",
  "provisioningState": "Succeeded",
  "registrationEnabled": false,
  "resolutionPolicy": "Default",
  "resourceGroup": "secu-ai-workshop",
  "type": "Microsoft.Network/privateDnsZones/virtualNetworkLinks",
  "virtualNetwork": {
    "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/spoke1vnet",
    "resourceGroup": "secu-ai-workshop"
  },
  "virtualNetworkLinkState": "Completed"
}


In [13]:
# Private DNS Zone Link 생성 (Onprem VNET)
!az network private-dns link vnet create \
    --name onpremvnetDnsLink \
    --resource-group $RESOURCE_GROUP \
    --zone-name $OPENAI_PRIVATE_DNS_ZONE \
    --virtual-network $ONPREM_VNET_NAME \
    --registration-enabled false

{
  "etag": "\"9c005c96-0000-0100-0000-67ef5abe0000\"",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/privateDnsZones/privatelink.openai.azure.com/virtualNetworkLinks/onpremvnetdnslink",
  "location": "global",
  "name": "onpremvnetdnslink",
  "provisioningState": "Succeeded",
  "registrationEnabled": false,
  "resolutionPolicy": "Default",
  "resourceGroup": "secu-ai-workshop",
  "type": "Microsoft.Network/privateDnsZones/virtualNetworkLinks",
  "virtualNetwork": {
    "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/onprem-vnet",
    "resourceGroup": "secu-ai-workshop"
  },
  "virtualNetworkLinkState": "Completed"
}


In [14]:
# Private DNS Zone Link 생성 (Hub VNET)
!az network private-dns link vnet create \
    --name hubvnetDnsLink \
    --resource-group $RESOURCE_GROUP \
    --zone-name $OPENAI_PRIVATE_DNS_ZONE \
    --virtual-network $HUB_VNET_NAME \
    --registration-enabled false

{
  "etag": "\"9c004d9b-0000-0100-0000-67ef5ae70000\"",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/privateDnsZones/privatelink.openai.azure.com/virtualNetworkLinks/hubvnetdnslink",
  "location": "global",
  "name": "hubvnetdnslink",
  "provisioningState": "Succeeded",
  "registrationEnabled": false,
  "resolutionPolicy": "Default",
  "resourceGroup": "secu-ai-workshop",
  "type": "Microsoft.Network/privateDnsZones/virtualNetworkLinks",
  "virtualNetwork": {
    "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/hubvnet",
    "resourceGroup": "secu-ai-workshop"
  },
  "virtualNetworkLinkState": "Completed"
}


In [15]:
OPENAI_PRIVATE_DNS_ZONE_ID=!az network private-dns zone show --name $OPENAI_PRIVATE_DNS_ZONE --resource-group $RESOURCE_GROUP --query id -o tsv
OPENAI_PRIVATE_DNS_ZONE_ID=OPENAI_PRIVATE_DNS_ZONE_ID[0].strip()
!echo $OPENAI_PRIVATE_DNS_ZONE_ID

/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/privateDnsZones/privatelink.openai.azure.com


In [16]:
# private dns record set create
!az network private-endpoint dns-zone-group create \
    -n "privatelink.openai.azure.com" \
    -g $RESOURCE_GROUP \
    --private-dns-zone $OPENAI_PRIVATE_DNS_ZONE \
    --endpoint-name $OPENAI_PE_NAME \
    --zone-name $OPENAI_PRIVATE_DNS_ZONE

{
  "etag": "W/\"e5190c30-44cb-4af9-a644-8d5b97606416\"",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/privateEndpoints/secu-ai-pe/privateDnsZoneGroups/privatelink.openai.azure.com",
  "name": "privatelink.openai.azure.com",
  "privateDnsZoneConfigs": [
    {
      "name": "privatelink.openai.azure.com",
      "privateDnsZoneId": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/privateDnsZones/privatelink.openai.azure.com",
      "recordSets": [
        {
          "fqdn": "secu-ai-account1.privatelink.openai.azure.com",
          "ipAddresses": [
            "10.1.1.4"
          ],
          "provisioningState": "Succeeded",
          "recordSetName": "secu-ai-account1",
          "recordType": "A",
          "ttl": 10
        }
      ]
    }
  ],
  "provisioningState": "Succeeded",
  "resourceGroup": "secu-ai-workshop"
}


In [17]:
# Azure OpenAI account에 모델 배포. GPT-4o 모델을 사용합니다.
!az cognitiveservices account deployment create \
    --name $OPENAI_NAME \
    --resource-group $RESOURCE_GROUP \
    --deployment-name gpt-4o \
    --model-name gpt-4o \
    --model-version "2024-08-06" \
    --sku-capacity "100" \
    --sku-name "GlobalStandard" \
    --model-format "OpenAI"

{
  "etag": "\"3d83c995-62ca-4b54-ab8f-f193c19c7967\"",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.CognitiveServices/accounts/secu-ai-account1/deployments/gpt-4o",
  "name": "gpt-4o",
  "properties": {
    "callRateLimit": null,
    "capabilities": {
      "assistants": "true",
      "chatCompletion": "true",
      "jsonSchemaResponse": "true",
      "maxContextToken": "128000",
      "maxOutputToken": "16384",
      "responses": "true"
    },
    "model": {
      "callRateLimit": null,
      "format": "OpenAI",
      "name": "gpt-4o",
      "source": null,
      "version": "2024-08-06"
    },
    "provisioningState": "Succeeded",
    "raiPolicyName": null,
    "rateLimits": [
      {
        "count": 1000.0,
        "dynamicThrottlingEnabled": null,
        "key": "request",
        "matchPatterns": null,
        "minCount": null,
        "renewalPeriod": 60.0
      },
      {
        "count": 100000.0,
        "dyn

#### 2.2 Key Vault (비밀/키 관리)

**Azure Portal:**

1.  "Create a resource"에서 "Key Vault"를 검색하여 선택하고 "Create"를 클릭합니다.
2.  다음 정보를 입력합니다.
    -   Subscription: 사용 중인 Azure 구독
    -   Resource group: "secu-ai-workshop"
    -   Key vault name: "secu-ai-kv" (고유한 이름)
    -   Region: "Korea Central"
    -   Pricing tier: Standard (필요에 따라 선택)
3.  "Review + create"를 클릭하고 "Create"를 클릭합니다.
4.  생성된 Key Vault에 OpenAI Key를 secret으로 저장합니다.

-   Key Vault의 "Secrets" 탭에서 "Generate/Import"를 클릭합니다.
    -   Upload options: Manual
    -   Name: openai-key
    -   Value: OpenAI API 키 입력 (다음 명령어로 확인: `az cognitiveservices account keys list --name $OPENAI_NAME --resource-group $RESOURCE_GROUP`)
    -   Content type: text
    -   Enabled: Yes
    -   Create

In [ ]:
# 변수 설정
KEY_VAULT_NAME="secu-ai-kv"

# Key Vault 생성
!az keyvault create \
    --name $KEY_VAULT_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION

# OpenAI API Key를 변수에 저장
OPENAI_API_KEY=$(!az cognitiveservices account keys list --name $OPENAI_NAME --resource-group $RESOURCE_GROUP --query key1 -o tsv)

# Key Vault에 OpenAI Key를 secret으로 저장
!az keyvault secret set \
    --vault-name $KEY_VAULT_NAME \
    --name openai-key \
    --value $OPENAI_API_KEY

#### 2.3 Azure API Management (Private Endpoint)

**Azure Portal:**

1.  "Create a resource"에서 "API Management"를 검색하여 선택하고 "Create"를 클릭합니다.
2.  다음 정보를 입력합니다.
    -   Subscription: 사용 중인 Azure 구독
    -   Resource group: "secu-ai-workshop"
    -   Region: "Korea Central"
    -   Name: "secu-ai-apim"
    -   Pricing tier: Developer (테스트 용도)
    -   Enable virtual network: On
    -   Virtual Network: spoke1vnet
    -   subnet: privateendpoint
3.  "Review + create" 를 클릭하고 "Create"를 클릭합니다.
4.  생성 후 apim의 private endpoint 생성
    -   apim의 "Private endpoint connections" 탭으로 이동합니다.
    -   "Private endpoint" 클릭
    -   Subscription: 사용 중인 Azure 구독
    -   Resource group: "secu-ai-workshop"
        -   Name: apim-pe
        -   Location: "Korea Central"
        -   Virtual Network: spoke1vnet
        -   Subnet: privateendpoint
        -   Integrate with private DNS zone: yes
        -   Private DNS Zone: privatelink.azure-api.net

In [21]:
# 변수 설정
APIM_NAME="secu-ai-apim-1"
APIM_PE_NAME="apim-pe"
APIM_PRIVATE_DNS_ZONE="privatelink.azure-api.net"

In [19]:
# APIM 생성을 위한 VNET Subnet ID 가져오기
SPOKE1_VNET_ID=!az network vnet show --name $SPOKE1_VNET_NAME --resource-group $RESOURCE_GROUP --query id -o tsv
SPOKE1_VNET_ID=SPOKE1_VNET_ID[0].strip()
SPOKE1_PE_SUBNET_ID=!az network vnet subnet show --name $SPOKE1_PE_SUBNET_NAME --resource-group $RESOURCE_GROUP --vnet-name $SPOKE1_VNET_NAME --query id -o tsv
SPOKE1_PE_SUBNET_ID=SPOKE1_PE_SUBNET_ID[0].strip()


In [22]:
# API Management 생성
!az apim create \
    --name $APIM_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --publisher-name "My Company" \
    --publisher-email "admin@mycompany.com" \
    --sku-name Developer

{
  "additionalLocations": null,
  "apiVersionConstraint": {
    "minApiVersion": null
  },
  "certificates": null,
  "createdAtUtc": "2025-04-04T04:45:43.923494+00:00",
  "customProperties": {
    "Microsoft.WindowsAzure.ApiManagement.Gateway.Protocols.Server.Http2": "False",
    "Microsoft.WindowsAzure.ApiManagement.Gateway.Security.Backend.Protocols.Ssl30": "False",
    "Microsoft.WindowsAzure.ApiManagement.Gateway.Security.Backend.Protocols.Tls10": "False",
    "Microsoft.WindowsAzure.ApiManagement.Gateway.Security.Backend.Protocols.Tls11": "False",
    "Microsoft.WindowsAzure.ApiManagement.Gateway.Security.Ciphers.TripleDes168": "False",
    "Microsoft.WindowsAzure.ApiManagement.Gateway.Security.Protocols.Ssl30": "False",
    "Microsoft.WindowsAzure.ApiManagement.Gateway.Security.Protocols.Tls10": "False",
    "Microsoft.WindowsAzure.ApiManagement.Gateway.Security.Protocols.Tls11": "False"
  },
  "developerPortalUrl": "https://secu-ai-apim-1.developer.azure-api.net",
  "disableGat

In [25]:
# APIM에 Private Endpoint 생성을 위한 변수 설정정
APIM_RESOURCE_ID=!az apim show --name $APIM_NAME --resource-group $RESOURCE_GROUP --query id -o tsv
APIM_RESOURCE_ID=APIM_RESOURCE_ID[0].strip()
!echo $APIM_RESOURCE_ID

/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.ApiManagement/service/secu-ai-apim-1


In [26]:
# Private Endpoint 생성
!az network private-endpoint create \
    --name $APIM_PE_NAME \
    --resource-group $RESOURCE_GROUP \
    --vnet-name $SPOKE1_VNET_NAME \
    --subnet $SPOKE1_PE_SUBNET_NAME \
    --private-connection-resource-id $APIM_RESOURCE_ID \
    --connection-name apim-connection \
    --location $LOCATION \
    --group-id Gateway 


{
  "customDnsConfigs": [
    {
      "fqdn": "secu-ai-apim-1.azure-api.net",
      "ipAddresses": [
        "10.1.1.5"
      ]
    }
  ],
  "customNetworkInterfaceName": "",
  "etag": "W/\"d11bec98-9063-4642-88b4-2a37e95f7000\"",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/privateEndpoints/apim-pe",
  "ipConfigurations": [],
  "location": "koreacentral",
  "manualPrivateLinkServiceConnections": [],
  "name": "apim-pe",
  "networkInterfaces": [
    {
      "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/networkInterfaces/apim-pe.nic.078e5bbf-d25d-4229-b476-b845f00f42f1",
      "resourceGroup": "secu-ai-workshop"
    }
  ],
  "privateLinkServiceConnections": [
    {
      "etag": "W/\"d11bec98-9063-4642-88b4-2a37e95f7000\"",
      "groupIds": [
        "Gateway"
      ],
      "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resour

In [ ]:
# APIM에 public endpoint을 사용하지 않도록 설정

!az apim update --name $APIM_NAME  --resource-group $RESOURCE_GROUP --public-network-access false --no-wait

{
  "additionalLocations": null,
  "apiVersionConstraint": {
    "minApiVersion": null
  },
  "certificates": null,
  "createdAtUtc": "2025-04-04T04:45:43.923494+00:00",
  "customProperties": {
    "Microsoft.WindowsAzure.ApiManagement.Gateway.Protocols.Server.Http2": "False",
    "Microsoft.WindowsAzure.ApiManagement.Gateway.Security.Backend.Protocols.Ssl30": "False",
    "Microsoft.WindowsAzure.ApiManagement.Gateway.Security.Backend.Protocols.Tls10": "False",
    "Microsoft.WindowsAzure.ApiManagement.Gateway.Security.Backend.Protocols.Tls11": "False",
    "Microsoft.WindowsAzure.ApiManagement.Gateway.Security.Ciphers.TripleDes168": "False",
    "Microsoft.WindowsAzure.ApiManagement.Gateway.Security.Protocols.Ssl30": "False",
    "Microsoft.WindowsAzure.ApiManagement.Gateway.Security.Protocols.Tls10": "False",
    "Microsoft.WindowsAzure.ApiManagement.Gateway.Security.Protocols.Tls11": "False"
  },
  "developerPortalUrl": "https://secu-ai-apim-1.developer.azure-api.net",
  "disableGat

In [28]:
# Private DNS Zone 생성 (privatelink.azure-api.net)
!az network private-dns zone create \
    --name $APIM_PRIVATE_DNS_ZONE \
    --resource-group $RESOURCE_GROUP

{
  "etag": "0a9cfc94-34e6-4c6a-9601-258e1d2f6ec7",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/privateDnsZones/privatelink.azure-api.net",
  "location": "global",
  "maxNumberOfRecordSets": 25000,
  "maxNumberOfVirtualNetworkLinks": 1000,
  "maxNumberOfVirtualNetworkLinksWithRegistration": 100,
  "name": "privatelink.azure-api.net",
  "numberOfRecordSets": 1,
  "numberOfVirtualNetworkLinks": 0,
  "numberOfVirtualNetworkLinksWithRegistration": 0,
  "provisioningState": "Succeeded",
  "resourceGroup": "secu-ai-workshop",
  "type": "Microsoft.Network/privateDnsZones"
}


In [29]:
# Private DNS Zone Link 생성 (Spoke1 VNET)
!az network private-dns link vnet create \
    --name spoke1vnetApimDnsLink \
    --resource-group $RESOURCE_GROUP \
    --zone-name $APIM_PRIVATE_DNS_ZONE \
    --virtual-network $SPOKE1_VNET_NAME \
    --registration-enabled false

{
  "etag": "\"9f00a2a6-0000-0100-0000-67ef746c0000\"",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/privateDnsZones/privatelink.azure-api.net/virtualNetworkLinks/spoke1vnetapimdnslink",
  "location": "global",
  "name": "spoke1vnetapimdnslink",
  "provisioningState": "Succeeded",
  "registrationEnabled": false,
  "resolutionPolicy": "Default",
  "resourceGroup": "secu-ai-workshop",
  "type": "Microsoft.Network/privateDnsZones/virtualNetworkLinks",
  "virtualNetwork": {
    "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/spoke1vnet",
    "resourceGroup": "secu-ai-workshop"
  },
  "virtualNetworkLinkState": "Completed"
}


In [30]:
# Private DNS Zone Link 생성 (Onprem VNET)
!az network private-dns link vnet create \
    --name onpremvnetApimDnsLink \
    --resource-group $RESOURCE_GROUP \
    --zone-name $APIM_PRIVATE_DNS_ZONE \
    --virtual-network $ONPREM_VNET_NAME \
    --registration-enabled false

{
  "etag": "\"9f0027ac-0000-0100-0000-67ef749b0000\"",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/privateDnsZones/privatelink.azure-api.net/virtualNetworkLinks/onpremvnetapimdnslink",
  "location": "global",
  "name": "onpremvnetapimdnslink",
  "provisioningState": "Succeeded",
  "registrationEnabled": false,
  "resolutionPolicy": "Default",
  "resourceGroup": "secu-ai-workshop",
  "type": "Microsoft.Network/privateDnsZones/virtualNetworkLinks",
  "virtualNetwork": {
    "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/onprem-vnet",
    "resourceGroup": "secu-ai-workshop"
  },
  "virtualNetworkLinkState": "Completed"
}


In [31]:
# Private DNS Zone Link 생성 (Hub VNET)
!az network private-dns link vnet create \
    --name hubvnetApimDnsLink \
    --resource-group $RESOURCE_GROUP \
    --zone-name $APIM_PRIVATE_DNS_ZONE \
    --virtual-network $HUB_VNET_NAME \
    --registration-enabled false

{
  "etag": "\"9f00e9b0-0000-0100-0000-67ef74c30000\"",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/privateDnsZones/privatelink.azure-api.net/virtualNetworkLinks/hubvnetapimdnslink",
  "location": "global",
  "name": "hubvnetapimdnslink",
  "provisioningState": "Succeeded",
  "registrationEnabled": false,
  "resolutionPolicy": "Default",
  "resourceGroup": "secu-ai-workshop",
  "type": "Microsoft.Network/privateDnsZones/virtualNetworkLinks",
  "virtualNetwork": {
    "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/hubvnet",
    "resourceGroup": "secu-ai-workshop"
  },
  "virtualNetworkLinkState": "Completed"
}


In [32]:
# private dns record set create
!az network private-endpoint dns-zone-group create \
    -n "privatelink.azure-api.net" \
    -g $RESOURCE_GROUP \
    --private-dns-zone $APIM_PRIVATE_DNS_ZONE \
    --endpoint-name $APIM_PE_NAME \
    --zone-name $OPENAI_PRIVATE_DNS_ZONE

{
  "etag": "W/\"0f3113e6-5e47-446b-be49-2ce48d81a625\"",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/privateEndpoints/apim-pe/privateDnsZoneGroups/privatelink.azure-api.net",
  "name": "privatelink.azure-api.net",
  "privateDnsZoneConfigs": [
    {
      "name": "privatelink.openai.azure.com",
      "privateDnsZoneId": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/privateDnsZones/privatelink.azure-api.net",
      "recordSets": [
        {
          "fqdn": "secu-ai-apim-1.privatelink.azure-api.net",
          "ipAddresses": [
            "10.1.1.5"
          ],
          "provisioningState": "Succeeded",
          "recordSetName": "secu-ai-apim-1",
          "recordType": "A",
          "ttl": 10
        }
      ]
    }
  ],
  "provisioningState": "Succeeded",
  "resourceGroup": "secu-ai-workshop"
}


#### 2.4 Log Analytics (APIM 등 모니터링)

**Azure Portal:**

1.  "Create a resource"에서 "Log Analytics workspace"를 검색하여 선택하고 "Create"를 클릭합니다.
2.  다음 정보를 입력합니다.
    -   Subscription: 사용 중인 Azure 구독
    -   Resource group: "secu-ai-workshop"
    -   Name: "secu-ai-log"
    -   Region: "Korea Central"
3.  "Review + create"를 클릭하고 "Create"를 클릭합니다.

In [34]:
# 변수 설정
LOG_ANALYTICS_NAME="secu-ai-log"

In [35]:
# Log Analytics Workspace 생성
!az monitor log-analytics workspace create \
    --name $LOG_ANALYTICS_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION

{
  "createdDate": "2025-04-01T06:57:54.7065528Z",
  "customerId": "f69555df-5c0e-4d7d-8f3b-28c9ddeb3422",
  "etag": "\"3400b5bb-0000-2500-0000-67ef9d6e0000\"",
  "features": {
    "enableLogAccessUsingOnlyResourcePermissions": true
  },
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.OperationalInsights/workspaces/secu-ai-log",
  "location": "koreacentral",
  "modifiedDate": "2025-04-04T08:50:54.812833Z",
  "name": "secu-ai-log",
  "provisioningState": "Succeeded",
  "publicNetworkAccessForIngestion": "Enabled",
  "publicNetworkAccessForQuery": "Enabled",
  "resourceGroup": "secu-ai-workshop",
  "retentionInDays": 30,
  "sku": {
    "lastSkuUpdate": "2025-04-01T06:57:54.7065528Z",
    "name": "PerGB2018"
  },
  "type": "Microsoft.OperationalInsights/workspaces",
  "workspaceCapping": {
    "dailyQuotaGb": -1.0,
    "dataIngestionStatus": "RespectQuota",
    "quotaNextResetTime": "2025-04-05T05:00:00Z"
  }
}


#### 2.5 APIM 진단 로그 설정

Azure Portal:

\[APIM 설정 가이드\]의 "Configure the Diagnostic Logs settings" 섹션을 참고합니다.

In [36]:
# Log Analytics Workspace ID 가져오기
LOG_ANALYTICS_WORKSPACE_ID=!az monitor log-analytics workspace show --name $LOG_ANALYTICS_NAME --resource-group $RESOURCE_GROUP --query id -o tsv
LOG_ANALYTICS_WORKSPACE_ID=LOG_ANALYTICS_WORKSPACE_ID[0].strip()
!echo $LOG_ANALYTICS_WORKSPACE_ID


/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.OperationalInsights/workspaces/secu-ai-log


In [53]:
LOGS="[{category:GatewayLogs,enabled:true}]"
METRICS="[{category:AllMetrics,enabled:true}]"

In [54]:
# APIM 진단 설정

!az monitor diagnostic-settings create \
    --name apim-diagnostics \
    --resource $APIM_NAME \
    --resource-group $RESOURCE_GROUP \
    --resource-type Microsoft.ApiManagement/service \
    --workspace $LOG_ANALYTICS_WORKSPACE_ID \
    --logs $LOGS \
    --metrics $METRICS

{
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourcegroups/secu-ai-workshop/providers/microsoft.apimanagement/service/secu-ai-apim-1/providers/microsoft.insights/diagnosticSettings/apim-diagnostics",
  "logs": [
    {
      "category": "GatewayLogs",
      "enabled": true,
      "retentionPolicy": {
        "days": 0,
        "enabled": false
      }
    }
  ],
  "metrics": [
    {
      "category": "AllMetrics",
      "enabled": true,
      "retentionPolicy": {
        "days": 0,
        "enabled": false
      },
      "timeGrain": "PT1M"
    }
  ],
  "name": "apim-diagnostics",
  "resourceGroup": "secu-ai-workshop",
  "type": "Microsoft.Insights/diagnosticSettings",
  "workspaceId": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.OperationalInsights/workspaces/secu-ai-log"
}


#### 2.6 APIM과 OpenAI Service 연결 및 Backend/API 설정

**APIM Named Value 설정 (Azure Portal):**

1.  API Management 인스턴스 (secu-ai-apim)로 이동합니다.

2.  "Named values"를 클릭하고 "+ Add"를 클릭합니다.

3.  다음 정보를 입력합니다.

    -   Name: openai-key-nv
    -   Display name: OpenAI Key
    -   Value type: Key vault
    -   Select from Key Vault:
        -   Subscription: 사용 중인 Azure 구독
        -   Key vault: secu-ai-kv 선택
        -   Secret: openai-key 선택
    -   Managed identity: System-assigned
4.  Create

**APIM Backend 설정 (Azure Portal):**

1.  API Management 인스턴스 (secu-ai-apim)로 이동합니다.
2.  "Backends"를 클릭하고 "+ Add"를 클릭합니다.
3.  다음 정보를 입력합니다.
    -   Backend id: openai-backend
    -   Backend type: Custom URL
    -   Runtime URL: `https://secu-ai-account.openai.azure.com/openai` (OpenAI 엔드포인트)
    -   Authorization:
        -   Add header:
            -   Name: api-key
            -   Value: {{openai-key-nv}} (Named Value 사용)
4.  "Create"를 클릭합니다.

**APIM API 설정 (Azure Portal):**

\[APIM 설정 가이드\]의 "API Import instructions" 섹션을 참고하여 OpenAI API를 APIM에 가져옵니다.

-   Completions OpenAPI spec URL: `https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/2023-05-15/inference.json`

Import 후 Operation을 Portal에서 테스트할 수 있습니다.


**Log Analytics 쿼리 예제:**

\[log analytics 가이드\]에 제공된 쿼리 예제를 참고하여 Log Analytics에서 APIM 요청 로그를 분석할 수 있습니다.

### 3단계: 테스트 환경 구성

#### 3.1 Virtual Machine (Onprem VNET에서 테스트 목적)

**Azure Portal:**

1.  "Create a resource"에서 "Virtual machine"을 검색하여 선택하고 "Create"를 클릭합니다.
2.  다음 정보를 입력합니다.
    -   Subscription: Azure 구독, Resource group: "secu-ai-workshop"
    -   Virtual machine name: "onprem-vm", Region: "Korea Central"
    -   Image: Ubuntu Server 20.04 LTS - Gen2
    -   Size: Standard_B2s (또는 적절한 크기)
    -   Username: 관리자 계정 이름
    -   Authentication type: SSH public key
        -   SSH public key source: Generate new key pair, Key pair name: onprem-vm-key
    -   Public inbound ports: None
    -   Networking: Virtual network: onprem-vnet, Subnet: onprem-subnet, Public IP: None
1.  "Review + create"를 클릭하고 "Create"를 클릭합니다.

In [64]:
# 변수 설정
VM_NAME="onprem-vm"
VM_IMAGE="Ubuntu2404"
VM_SIZE="Standard_B2s" # 필요에 따라 변경
VM_ADMIN_USERNAME="adminuser"
VM_ADMIN_PASSWORD="adminpassword1!" # 필요에 따라 변경

In [ ]:
# VM 생성 (패스워드 인증 방식)

!az vm create \
    --name $VM_NAME \
    --resource-group $RESOURCE_GROUP \
    --image $VM_IMAGE \
    --size $VM_SIZE \
    --vnet-name $ONPREM_VNET_NAME \
    --subnet $ONPREM_SUBNET_NAME \
    --admin-username $VM_ADMIN_USERNAME \
    --admin-password $VM_ADMIN_PASSWORD \
    --authentication-type password

{
  "fqdns": "",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Compute/virtualMachines/onprem-vm",
  "location": "koreacentral",
  "macAddress": "00-22-48-05-27-4C",
  "powerState": "VM running",
  "privateIpAddress": "192.168.1.4",
  "publicIpAddress": "4.218.16.29",
  "resourceGroup": "secu-ai-workshop",
  "zones": ""
}


### 3.2 Bastion (Onprem VNET)
**Azure Portal:**

1.  Onprem VNET ("onprem-vnet") - "Settings" - "Bastion" - "Configure manually"
2.  정보 입력:
    * Name: onprem-bastion
    * Tier: Basic
    * Instance count: 1
    * Virtual Network: onprem-vnet
    * subnet: AzureBastionSubnet를 192.168.2.0/24로 생성
    * Public IP Address: 새로 생성 후 이름 입력

In [66]:
# Bastion Subnet 생성
!az network vnet subnet create \
    --name AzureBastionSubnet \
    --resource-group $RESOURCE_GROUP \
    --vnet-name $ONPREM_VNET_NAME \
    --address-prefix 192.168.2.0/24

{
  "addressPrefix": "192.168.2.0/24",
  "delegations": [],
  "etag": "W/\"e374b3f4-4a4a-4351-9f4c-82388fd0f397\"",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/virtualNetworks/onprem-vnet/subnets/AzureBastionSubnet",
  "name": "AzureBastionSubnet",
  "privateEndpointNetworkPolicies": "Disabled",
  "privateLinkServiceNetworkPolicies": "Enabled",
  "provisioningState": "Succeeded",
  "resourceGroup": "secu-ai-workshop",
  "type": "Microsoft.Network/virtualNetworks/subnets"
}


In [69]:
# Bastion Public IP 생성
BASTION_PIP_NAME="onprem-bastion-pip"
BASTION_NAME="onprem-bastion"

In [68]:
# Bastion Public IP 생성
!az network public-ip create \
    --name $BASTION_PIP_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --sku Standard

{
  "publicIp": {
    "ddosSettings": {
      "protectionMode": "VirtualNetworkInherited"
    },
    "etag": "W/\"079f0545-a4bb-4316-96a2-8e209624ebbf\"",
    "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/publicIPAddresses/onprem-bastion-pip",
    "idleTimeoutInMinutes": 4,
    "ipAddress": "4.218.22.133",
    "ipTags": [],
    "location": "koreacentral",
    "name": "onprem-bastion-pip",
    "provisioningState": "Succeeded",
    "publicIPAddressVersion": "IPv4",
    "publicIPAllocationMethod": "Static",
    "resourceGroup": "secu-ai-workshop",
    "resourceGuid": "78631048-6f55-4e02-93fd-5bf0fcec5d4b",
    "sku": {
      "name": "Standard",
      "tier": "Regional"
    },
    "type": "Microsoft.Network/publicIPAddresses"
  }
}


In [70]:
# Bastion 생성

!az network bastion create \
    --name $BASTION_NAME \
    --public-ip-address $BASTION_PIP_NAME \
    --resource-group $RESOURCE_GROUP \
    --vnet-name $ONPREM_VNET_NAME \
    --location $LOCATION

{
  "disableCopyPaste": false,
  "dnsName": "bst-e362a3de-a8e3-46ec-85dd-e37b097c583e.bastion.azure.com",
  "enableFileCopy": false,
  "enableIpConnect": false,
  "enableKerberos": false,
  "enableSessionRecording": false,
  "enableShareableLink": false,
  "enableTunneling": false,
  "etag": "W/\"051a7542-42af-49ae-8edc-3483181240a6\"",
  "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/bastionHosts/onprem-bastion",
  "ipConfigurations": [
    {
      "etag": "W/\"051a7542-42af-49ae-8edc-3483181240a6\"",
      "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/resourceGroups/secu-ai-workshop/providers/Microsoft.Network/bastionHosts/onprem-bastion/bastionHostIpConfigurations/bastion_ip_config",
      "name": "bastion_ip_config",
      "privateIPAllocationMethod": "Dynamic",
      "provisioningState": "Succeeded",
      "publicIPAddress": {
        "id": "/subscriptions/66a8cf06-a653-4823-a2fd-95ab22ac4509/reso

### 3.3 OpenAI API 호출 테스트

1.  Bastion을 통해 On-Prem VM에 접속 (Azure Portal):
    -   onprem-vm - "Connect" - "Bastion" 선택
    -   username, Authentication Type(from Azure Bastion), Private Key from Local File을 선택하고 다운로드 받은 키 업로드
    -   Connect
2.  On-Prem VM에서 APIM Private Endpoint를 통해 OpenAI API 호출 (curl 사용):

In [71]:
!az apim show \
  --name $APIM_NAME \
  --resource-group $RESOURCE_GROUP \
  --query "gatewayUrl" \
  --output tsv


https://secu-ai-apim-1.azure-api.net


In [75]:
!az apim subscription list-keys

ERROR: 'subscription' is misspelled or not recognized by the system.

Examples from AI knowledge base:
https://aka.ms/cli_ref
Read more about the command in reference docs


In [74]:
!az apim subscription list \
  --resource-group $RESOURCE_GROUP \
  --service-name $APIM_NAME \
  --query "[0].id" \
  --output tsv

ERROR: 'subscription' is misspelled or not recognized by the system.

Examples from AI knowledge base:
https://aka.ms/cli_ref
Read more about the command in reference docs


In [ ]:
curl --request POST \
    --url "https://secu-ai-apim-1.azure-api.net/openapis/deployments/gpt-4o/chat/completions?api-version=2024-02-01" \
    --header "api-key: 8c0a3292cfea444485a0eebc10add7f7" \
    --header "Content-Type: application/json" \
    --data '{"messages":[{"role":"system","content":"You are a helpful assistant that provides detailed information about Microsoft Azure."},{"role":"user","content":"How can I leverage Microsoft Azure across different business scenarios?"}]}'

In [ ]:
curl --request POST --url "https://secu-ai-apim-1.azure-api.net/apicall/deployments/gpt-4o/chat/completions?api-version=2024-02-01" --header "api-key: d65e4cd35b034675970a652e34a0b9c7" --header "Content-Type: application/json" --data '{"messages":[{"role":"system","content":"You are a helpful assistant that provides detailed information about Microsoft Azure."},{"role":"user","content":"How can I leverage Microsoft Azure across different business scenarios?"}]}'

In [ ]:
# APIM 엔드포인트 확인 (Azure Portal에서 확인)
APIM_ENDPOINT="https://secu-ai-apim.privatelink.azure-api.net/"

# OpenAI API 호출 (예: Completions)
curl $APIM_ENDPOINT/openai/deployments/gpt-4o/completions?api-version=2023-05-15 \
  -H "Content-Type: application/json" \
  -H "api-key: $(az keyvault secret show --vault-name $KEY_VAULT_NAME --name openai-key --query value -o tsv)" \
  -d '{
    "prompt": "The quick brown fox jumps over the lazy dog.",
    "max_tokens": 5
  }'

**curl 요청 설명:**

    -   $APIM_ENDPOINT/openai/deployments/gpt-4o/completions?api-version=2023-05-15: APIM을 통해 OpenAI Completions API 호출 URL.
    -   -H "Content-Type: application/json": 요청 본문 JSON 형식.
    -   -H "api-key: $(az keyvault secret show ...)": Key Vault에서 가져온 OpenAI API 키를 헤더에 추가.
    -   -d '{ ... }': API 요청 본문 (prompt, max_tokens 등).

### 3.4 Log Analytics에서 APIM을 통과한 OpenAI API 호출 결과 모니터링

**Azure Portal에서 Log Analytics 작업 영역(secu-ai-log)으로 이동**

    -   "Logs" 블레이드 선택
    -   다음 쿼리 실행하여 APIM을 통해 들어온 OpenAI API 호출 결과 및 사용 토큰량 확인

In [ ]:
ApiManagementGatewayLogs
| where tolower(OperationId) in ('completions_create','chatcompletions_create')
| where ResponseCode == '200'
| extend modelkey = substring(parse_json(BackendResponseBody)['model'], 0, indexof(parse_json(BackendResponseBody)['model'], '-', 0, -1, 2))
| extend model = tostring(parse_json(BackendResponseBody)['model'])
| extend prompttokens = parse_json(parse_json(BackendResponseBody)['usage'])['prompt_tokens']
| extend completiontokens = parse_json(parse_json(BackendResponseBody)['usage'])['completion_tokens']
| extend totaltokens = parse_json(parse_json(BackendResponseBody)['usage'])['total_tokens']
| extend ip = CallerIpAddress
| where model != ''
| summarize
    sum(todecimal(prompttokens)),
    sum(todecimal(completiontokens)),
    sum(todecimal(totaltokens)),
    avg(todecimal(totaltokens))
    by ip, model


In [ ]:
ApiManagementGatewayLogs
| where tolower(OperationId) in ('completions_create','chatcompletions_create')
| where ResponseCode  == '200'
| extend model = tostring(parse_json(BackendResponseBody)['model'])
| extend prompttokens = parse_json(parse_json(BackendResponseBody)['usage'])['prompt_tokens']
| extend prompttext = substring(parse_json(parse_json(BackendResponseBody)['choices'])[0], 0, 100)

**결과 확인**: curl 명령어가 정상 실행되면 OpenAI API로부터 응답(JSON)을 받음. 생성된 텍스트, 사용된 토큰 수 등 확인.

**이것으로 Azure Hub & Spoke 아키텍처를 활용하여 On-premise 환경에서 Azure OpenAI Service를 Private Endpoint를 통해 안전하게 호출하는 워크샵 구성을 완료했습니다.**